In [19]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [42]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [43]:
nasa_news_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_news_url)
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

In [23]:
news_title = news_soup.find_all('div', class_='content_title')[0].text
news = news_soup.find_all('div', class_='article_teaser_body')[0].text

print(news_title)
print("--------------------------------------------------------------------")
print(news)

Mars Now
--------------------------------------------------------------------
The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative.


In [24]:
mars_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
jpl_img_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16225_hires.jpg'

browser.visit(mars_url)

html = browser.html

soupy_img = BeautifulSoup(html, 'html.parser')

In [25]:
#scrape the url that links to the featured image using beautiful soup variable
img_path = soupy_img.find_all('img')[3]["src"]
featured_image_url = mars_url + img_path
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA24272-640x350.jpg


In [26]:
#get some cool facts
facts_link = 'https://space-facts.com/mars/'
fact_out = pd.read_html(facts_link)
fact_out

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [33]:
#Mars Hemisphere
usgs_url = 'https://astrogeology.usgs.gov'
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(hemispheres_url)

hemispheres_html = browser.html

hemispheres_soup = BeautifulSoup(hemispheres_html, 'html.parser')

In [38]:
every_mars_hemisphere = hemispheres_soup.find('div', class_='collapsible results')
mars_hemispheres = every_mars_hemisphere.find_all('div', class_='item')

hemisphere_image_urls = []

#loop through hemisphere data
for i in mars_hemispheres:
  
    hemisphere = i.find('div', class_="description")
    title = hemisphere.h3.text
    
    #scrape the img url from the page
    hemisphere_url = hemisphere.a["href"]    
    browser.visit(usgs_url + hemisphere_url)
    
    html = browser.html
    img_soup = BeautifulSoup(html, 'html.parser')
    
    image_link = img_soup.find('div', class_='downloads')
    
    #store 'li' and the 'href' from html code from webpage into a variable to invoke it later
    img_url = image_link.find('li').a['href']

    #store title and url in a dictionary
    image_dict = {}
    image_dict['title'] = title
    image_dict['img_url'] = img_url
    hemisphere_image_urls.append(image_dict)

print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [41]:
#Create a dataframe to visualize the values from the facts link

fact_df = fact_out[2]
fact_df

#name the columns to better names
fact_df.columns = ["Description", "Mars"]
fact_df

,Description,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers
